In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:41125 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_8617/'

In [5]:
sst = xr.open_dataset('/g/data/e14/erd561/NOAA_OI_SST_V2/' + 'sst.mnmean.nc').\
                sst.\
                groupby('time.year').\
                mean('time', skipna=True).\
                sel(year=slice(1986,2017))
print(sst)

mask = xr.open_dataset('/g/data/e14/erd561/NOAA_OI_SST_V2/' + 'lsmask.nc').\
                mask
sst = sst.where(mask==1, np.nan)
print(sst)

sst = sst.mean('lon').mean('lat')
print(sst)

sst_global = sst.squeeze()
print(sst_global)

<xarray.DataArray 'sst' (year: 32, lat: 180, lon: 360)>
array([[[-1.7900003, -1.7900003, -1.7900003, ..., -1.7900003,
         -1.7900003, -1.7900003],
        [-1.7900003, -1.7900003, -1.7900003, ..., -1.7900003,
         -1.7900003, -1.7900003],
        [-1.7683333, -1.7675   , -1.7666665, ..., -1.7808336,
         -1.7808336, -1.7800001],
        ...,
        [-1.7900003, -1.7900003, -1.7900003, ..., -1.7900003,
         -1.7900003, -1.7900003],
        [-1.7900003, -1.7900003, -1.7900003, ..., -1.7900003,
         -1.7900003, -1.7900003],
        [-1.7900003, -1.7900003, -1.7900003, ..., -1.7900003,
         -1.7900003, -1.7900003]],

       [[-1.7900003, -1.7900003, -1.7900003, ..., -1.7900003,
         -1.7900003, -1.7900003],
        [-1.7900003, -1.7900003, -1.7900003, ..., -1.7900003,
         -1.7900003, -1.7900003],
        [-1.7783335, -1.7750001, -1.7750001, ..., -1.7891668,
         -1.7891668, -1.7891668],
        ...,
        [-1.7900003, -1.7900003, -1.7900003, ..., -1

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [12]:
%%time
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
# time_units = exptdata.exptdict[ekey]['time_units']
n_files = exptdata.exptdict[ekey]['n_files']
offset = exptdata.exptdict[ekey]['offset']

print(expt)

temp_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp',
                                     n=n_files,
                                     offset=offset).\
mean('yt_ocean').\
mean('xt_ocean').\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_1_mod_month)

temp_z0_1_mod_month.load()
print(temp_z0_1_mod_month)

temp_z0_1_mod_all = temp_z0_1_mod_month.groupby('time.year').mean('time')
print(temp_z0_1_mod_all)

temp_z0_1_mod_global = temp_z0_1_mod_all.sel(year=slice(1986,2017))
print(temp_z0_1_mod_global)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 60)>
dask.array<sub, shape=(60,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    st_ocean  float64 1.152
  * time      (time) object 1958-07-03 12:00:00 ... 2017-07-02 12:00:00
<xarray.DataArray 'temp' (time: 60)>
array([14.899017 , 14.831787 , 14.817078 , 14.763367 , 14.741089 ,
       14.780487 , 14.697113 , 14.692444 , 14.716064 , 14.657898 ,
       14.6586   , 14.7960205, 14.70755  , 14.626526 , 14.8020935,
       14.807587 , 14.6588745, 14.650452 , 14.674347 , 14.827209 ,
       14.743591 , 14.851288 , 14.873749 , 14.794067 , 14.814636 ,
       14.945831 , 14.793274 , 14.766418 , 14.794006 , 14.948761 ,
       14.859894 , 14.802368 , 14.932648 , 14.955292 , 14.903442 ,
       14.88797  , 14.860962 , 14.94162  , 14.862518 , 15.001221 ,
       15.091003 , 14.837372 , 14.852356 , 14.935455 , 14.979187 ,
       14.987427 , 14.959808 , 15.010712 , 15.015015 , 14.955627 ,
       14.9028015, 15.007111 , 15.058533 , 14.937958 , 15.

In [13]:
%%time
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
# time_units = exptdata.exptdict[ekey]['time_units']
n_files = exptdata.exptdict[ekey]['n_files']
offset = exptdata.exptdict[ekey]['offset']

print(expt)

temp_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp',
                                     n=n_files,
                                     offset=offset).\
mean('yt_ocean').\
mean('xt_ocean').\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_025_mod_month)

temp_z0_025_mod_month.load()
print(temp_z0_025_mod_month)

temp_z0_025_mod_all = temp_z0_025_mod_month.groupby('time.year').mean('time')
print(temp_z0_025_mod_all)

temp_z0_025_mod_global = temp_z0_025_mod_all.sel(year=slice(1986,2017))
print(temp_z0_025_mod_global)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 60)>
dask.array<sub, shape=(60,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    st_ocean  float64 1.152
  * time      (time) object 1958-07-01 12:00:00 ... 2017-06-30 12:00:00
<xarray.DataArray 'temp' (time: 60)>
array([11.098541 , 11.046997 , 11.032867 , 11.022675 , 10.97641  ,
       10.981598 , 10.9375   , 10.925995 , 10.915802 , 10.925873 ,
       10.901123 , 10.97467  , 10.939301 , 10.919159 , 11.005615 ,
       11.016876 , 10.936584 , 10.944885 , 10.93808  , 11.055542 ,
       10.990234 , 11.057068 , 11.086365 , 11.027557 , 10.994415 ,
       11.104462 , 11.026367 , 11.019501 , 11.012695 , 11.110413 ,
       11.088837 , 11.06308  , 11.141113 , 11.136017 , 11.1033325,
       11.077789 , 11.06015  , 11.13031  , 11.076691 , 11.178833 ,
       11.23114  , 11.080078 , 11.084534 , 11.144165 , 11.157379 ,
       11.16922  , 11.148499 , 11.189148 , 11.201447 , 11.165009 ,
       11.117493 , 11.173279 , 11.218201 , 11.162018 , 11.

In [9]:
%%time
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
# time_units = exptdata.exptdict[ekey]['time_units']
n_files = exptdata.exptdict[ekey]['n_files']
offset = exptdata.exptdict[ekey]['offset']
print(expt)

# temp_z0_01_mod_month = cc.querying.getvar(expt=expt,
#                                      session=session,
#                                      ncfile='ocean.nc',
#                                      variable='temp',
#                                      n=n_files,
#                                      offset=offset).\
# sel(xt_ocean=arange(-280,80,1), method='nearest').\
# sel(yt_ocean=arange(-81,90,1), method='nearest').\
# sel(st_ocean=0, method='nearest')-273.15

temp_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp',
                                     n=n_files,
                                     offset=offset).\
mean('xt_ocean').\
mean('yt_ocean').\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_01_mod_month)

temp_z0_01_mod_month.load()
print(temp_z0_01_mod_month)

temp_z0_01_mod_all = temp_z0_01_mod_month.groupby('time.year').mean('time')
print(temp_z0_01_mod_all)

temp_z0_01_mod_global = temp_z0_01_mod_all.sel(year=slice(1986,2017))
print(temp_z0_01_mod_global)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 396)>
dask.array<sub, shape=(396,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    st_ocean  float64 0.5413
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
<xarray.DataArray 'temp' (time: 396)>
array([ 9.366333 ,  9.339081 ,  9.153839 ,  9.028534 ,  9.124542 ,
        9.589722 , 10.3471985, 10.736084 , 10.4131775,  9.770264 ,
        9.257782 ,  9.162445 ,  9.283447 ,  9.28833  ,  9.163818 ,
        9.037872 ,  9.158295 ,  9.626831 , 10.351532 , 10.736053 ,
       10.406586 ,  9.742859 ,  9.305542 ,  9.183716 ,  9.3255005,
        9.353302 ,  9.217346 ,  9.086609 ,  9.19455  ,  9.6718445,
       10.410217 , 10.830566 , 10.495087 ,  9.810516 ,  9.339935 ,
        9.256897 ,  9.458527 ,  9.449768 ,  9.208313 ,  9.078705 ,
        9.170929 ,  9.66687  , 10.434967 , 10.773743 , 10.462891 ,
        9.774811 ,  9.252197 ,  9.143494 ,  9.367706 ,  9.320801 ,
        9.170959 ,  9.064362 ,  9.201752 ,  9.680664 ,

In [14]:
# temp_z0_global_oisst_xr = xr.DataArray(sst_global.transpose('year'), name='temp_z0_global_oisst',
#                         coords=[sst_global.year], 
#                         dims=['year'])
# print(temp_z0_global_oisst_xr)
# temp_z0_global_oisst_xr.to_netcdf(output_path + 'temp_z0_global_oisst.nc')


temp_z0_1_global_mod_xr = xr.DataArray(temp_z0_1_mod_global.transpose('year'), name='temp_z0_1_global_mod',
                        coords=[temp_z0_1_mod_global.year], 
                        dims=['year'])
print(temp_z0_1_global_mod_xr)
temp_z0_1_global_mod_xr.to_netcdf(output_path + 'temp_z0_1_global_mod.nc')


temp_z0_025_global_mod_xr = xr.DataArray(temp_z0_025_mod_global.transpose('year'), name='temp_z0_025_global_mod',
                        coords=[temp_z0_025_mod_global.year], 
                        dims=['year'])
print(temp_z0_025_global_mod_xr)
temp_z0_025_global_mod_xr.to_netcdf(output_path + 'temp_z0_025_global_mod.nc')


# temp_z0_01_global_mod_xr = xr.DataArray(temp_z0_01_mod_global.transpose('year'), name='temp_z0_01_global_mod',
#                         coords=[temp_z0_01_mod_global.year], 
#                         dims=['year'])
# print(temp_z0_01_global_mod_xr)
# temp_z0_01_global_mod_xr.to_netcdf(output_path + 'temp_z0_01_global_mod.nc')

<xarray.DataArray 'temp_z0_1_global_mod' (year: 32)>
array([14.794006 , 14.948761 , 14.859894 , 14.802368 , 14.932648 ,
       14.955292 , 14.903442 , 14.88797  , 14.860962 , 14.94162  ,
       14.862518 , 15.001221 , 15.091003 , 14.837372 , 14.852356 ,
       14.935455 , 14.979187 , 14.987427 , 14.959808 , 15.010712 ,
       15.015015 , 14.955627 , 14.9028015, 15.007111 , 15.058533 ,
       14.937958 , 15.00412  , 15.029053 , 15.057739 , 15.164337 ,
       15.263367 , 15.150818 ], dtype=float32)
Coordinates:
  * year     (year) int64 1986 1987 1988 1989 1990 ... 2013 2014 2015 2016 2017
<xarray.DataArray 'temp_z0_025_global_mod' (year: 32)>
array([11.012695 , 11.110413 , 11.088837 , 11.06308  , 11.141113 ,
       11.136017 , 11.1033325, 11.077789 , 11.06015  , 11.13031  ,
       11.076691 , 11.178833 , 11.23114  , 11.080078 , 11.084534 ,
       11.144165 , 11.157379 , 11.16922  , 11.148499 , 11.189148 ,
       11.201447 , 11.165009 , 11.117493 , 11.173279 , 11.218201 ,
       11.16201

In [11]:
temp_z0_01_mod_global

<xarray.DataArray 'temp' (year: 32)>
array([9.607074 , 9.666031 , 9.656268 , 9.6588335, 9.742958 , 9.748779 ,
       9.659302 , 9.656674 , 9.636317 , 9.722008 , 9.688583 , 9.785457 ,
       9.8120985, 9.679515 , 9.686218 , 9.726254 , 9.76842  , 9.806827 ,
       9.808014 , 9.842982 , 9.839398 , 9.791463 , 9.735082 , 9.783615 ,
       9.832782 , 9.76875  , 9.78447  , 9.803042 , 9.86972  , 9.859734 ,
       9.977409 , 9.948009 ], dtype=float32)
Coordinates:
    st_ocean  float64 0.5413
  * year      (year) int64 1986 1987 1988 1989 1990 ... 2013 2014 2015 2016 2017